In [8]:

import fitz  # PyMuPDF
import os
pdf_path = r"C:\Users\codek\Desktop\con_safe1\회의\4. 시스템서포트PDF-d.pdf"
#pdf 열기
doc = fitz.open(pdf_path)

In [10]:
import fitz  # PyMuPDF\
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageEnhance
import os

def extract_images():
    try:
        # 사용자 입력 값 가져오기
        pages_per_position = int(entry_pages_per_position.get())  # 위치당 페이지 수
        target_pages = list(map(int, entry_target_pages.get().split(',')))  # 출력할 페이지 번호
        pdf_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])  # PDF 파일 선택
        if not pdf_path:
            messagebox.showerror("오류", "PDF 파일을 선택하세요.")
            return
        
        # 이미지 저장 폴더 선택
        output_folder = filedialog.askdirectory()
        if not output_folder:
            messagebox.showerror("오류", "이미지를 저장할 폴더를 선택하세요.")
            return
        
        # PDF 열기
        doc = fitz.open(pdf_path)

        # 페이지 추출 및 저장
        for i in range(len(doc) // pages_per_position):
            for page_offset in target_pages:
                page_num = (i * pages_per_position) + (page_offset - 1)  # 페이지 번호 계산
                if page_num < len(doc):
                    page = doc[page_num]
                    pix = page.get_pixmap(dpi=300, colorspace=fitz.csRGB)

                    # 저장 경로
                    output_path = os.path.join(output_folder, f"page_{page_num + 1}.png")
                    pix.save(output_path)
        
        messagebox.showinfo("완료", f"이미지 추출 완료!\n저장 폴더: {output_folder}")
    except Exception as e:
        messagebox.showerror("오류", f"오류 발생: {str(e)}")

# Tkinter GUI 생성
root = tk.Tk()
root.title("PDF 특정 페이지 이미지 추출")

# 설명 레이블
tk.Label(root, text="위치당 페이지 수 (예: 4)").grid(row=0, column=0, padx=5, pady=5)
entry_pages_per_position = tk.Entry(root)
entry_pages_per_position.grid(row=0, column=1, padx=5, pady=5)

tk.Label(root, text="출력할 페이지 번호 (예: 2,4)").grid(row=1, column=0, padx=5, pady=5)
entry_target_pages = tk.Entry(root)
entry_target_pages.grid(row=1, column=1, padx=5, pady=5)

# 실행 버튼
tk.Button(root, text="PDF 이미지 추출", command=extract_images).grid(row=2, column=0, columnspan=2, pady=10)

# Tkinter GUI 실행
root.mainloop()

# PDF 파일 경로
pdf_path = r"C:\Users\codek\Desktop\con_safe1\회의\5. 파이프서포트PDF-d.pdf"

# 이미지 저장 폴더
output_folder = r"C:\Users\codek\Desktop\con_safe1\회의\이미지_최적화5"
os.makedirs(output_folder, exist_ok=True)  # 폴더 생성

# PDF 열기
doc = fitz.open(pdf_path)

# 최적화 설정
dpi = 200  # 해상도
zoom_x = 4.0  # X축 확대 배율
zoom_y = 4.0  # Y축 확대 배율
mat = fitz.Matrix(zoom_x, zoom_y)  # 확대 매트릭스 설정


# 4쪽 단위로 2번째와 4번째 페이지 저장
for i in range(len(doc) // 4):
    # 페이지 번호 계산 (0 기반 인덱스)
    page_2 = (i * 4) + 1  # 4쪽 단위의 2번째 페이지
    page_4 = (i * 4) + 3  # 4쪽 단위의 4번째 페이지

    # 각 페이지를 이미지로 저장
    for page_num in [page_2, page_4]:
        page = doc[page_num]
    
         # 페이지 크기 확인 (퍼센트 계산에 사용)
        page_size = page.rect
        width = page_size.width  # 페이지 가로 길이 (포인트 단위)
        height = page_size.height  # 페이지 세로 길이 (포인트 단위)

        # **퍼센트 단위로 영역 설정 (예: 페이지의 상단 25% 영역)**
        x1 = 0  # 좌측 끝
        y1 = 0  # 상단
        x2 = width  # 우측 끝
        y2 = height # 상단 긑  / *0.25
        crop_rect = fitz.Rect(x1, y1, x2, y2)

        # **특정 영역 렌더링**
        pix = page.get_pixmap(matrix=mat, dpi=dpi, colorspace=fitz.csRGB, clip=crop_rect)

        # 이미지 저장 경로
        output_path = os.path.join(output_folder, f"page_{page_num + 1}_cropped.png")
        pix.save(output_path)

        # **이미지 대비 강화 (Pillow 사용)**
        # 저장된 이미지를 읽고 대비를 강화
        image = Image.open(output_path)
        # 대비 및 선명도 강화
        contrast_enhancer = ImageEnhance.Contrast(image)
        image = contrast_enhancer.enhance(1.8)  # 대비 강화
        sharpness_enhancer = ImageEnhance.Sharpness(image)
        image = sharpness_enhancer.enhance(2.0)  # 선명도 강화

        # 강화된 이미지 저장
        image.save(output_path)

print("PDF 페이지를 최적화된 이미지로 저장 완료!")


PDF 페이지를 최적화된 이미지로 저장 완료!


In [20]:
import tkinter as tk
from tkinter import filedialog, messagebox
import fitz  # PyMuPDF
import os

# Tkinter GUI 생성
root = tk.Tk()
root.title("PDF 특정 페이지 이미지 추출")

# 설명 레이블
tk.Label(root, text="구역당 페이지 수 ").grid(row=0, column=0, padx=5, pady=5)
entry_pages_per_position = tk.Entry(root)
entry_pages_per_position.grid(row=0, column=1, padx=5, pady=5)

tk.Label(root, text="출력할 페이지 번호 (예: 2,4)").grid(row=1, column=0, padx=5, pady=5)
entry_target_pages = tk.Entry(root)
entry_target_pages.grid(row=1, column=1, padx=5, pady=5)

# 최적화 설정
zoom_x = 4.0  # X축 확대 배율
zoom_y = 4.0  # Y축 확대 배율
mat = fitz.Matrix(zoom_x, zoom_y)  # 확대 매트릭스 설정
#xy축 출력될 곳 비율설정
# DPI 입력
tk.Label(root, text="DPI (해상도)").grid(row=2, column=0, padx=5, pady=5)
entry_dpi = tk.Entry(root)
entry_dpi.insert(0, "200")  # 기본값
entry_dpi.grid(row=2, column=1, padx=5, pady=5)

# XY 시작/끝 비율 입력
tk.Label(root, text="X 시작 비율 (0~1)").grid(row=3, column=0, padx=5, pady=5)
entry_x_start = tk.Entry(root)
entry_x_start.insert(0, "0.032")  # 기본값
entry_x_start.grid(row=3, column=1, padx=5, pady=5)

tk.Label(root, text="Y 시작 비율 (0~1)").grid(row=4, column=0, padx=5, pady=5)
entry_y_start = tk.Entry(root)
entry_y_start.insert(0, "0.032")  # 기본값
entry_y_start.grid(row=4, column=1, padx=5, pady=5)

tk.Label(root, text="X 끝 비율 (0~1)").grid(row=5, column=0, padx=5, pady=5)
entry_x_end = tk.Entry(root)
entry_x_end.insert(0, "0.968")  # 기본값
entry_x_end.grid(row=5, column=1, padx=5, pady=5)

tk.Label(root, text="Y 끝 비율 (0~1)").grid(row=6, column=0, padx=5, pady=5)
entry_y_end = tk.Entry(root)
entry_y_end.insert(0, "0.968")  # 기본값
entry_y_end.grid(row=6, column=1, padx=5, pady=5)

# 실행 버튼
def extract_images():
    try:
        # 사용자 입력 값 가져오기
        pages_per_position = int(entry_pages_per_position.get())
        target_pages = list(map(int, entry_target_pages.get().split(',')))
        pdf_path = filedialog.askopenfilename(title="PDF 파일을 선택하세요", filetypes=[("PDF Files", "*.pdf")])
        if not pdf_path:
            messagebox.showerror("오류", "PDF 파일을 선택하세요.")
            return
        output_folder = filedialog.askdirectory(title="이미지를 저장할 폴더를 선택하세요")
        if not output_folder:
            messagebox.showerror("오류", "이미지를 저장할 폴더를 선택하세요.")
            return
        
        # PDF 열기
        doc = fitz.open(pdf_path)
        pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]

        # 입력값 검증
        dpi = int(entry_dpi.get())
        x_start = float(entry_x_start.get())
        y_start = float(entry_y_start.get())
        x_end = float(entry_x_end.get())
        y_end = float(entry_y_end.get())

        if not (0 <= x_start <= 1 and 0 <= y_start <= 1 and 0 <= x_end <= 1 and 0 <= y_end <= 1):
            raise ValueError("XY 비율 값은 0과 1 사이여야 합니다.")
        if x_start >= x_end or y_start >= y_end:
            raise ValueError("시작 비율은 끝 비율보다 작아야 합니다.")

        # 페이지 추출 및 저장
        for i in range(len(doc) // pages_per_position):
            for page_offset in target_pages:
                page_num = (i * pages_per_position) + (page_offset - 1)
                if page_num < len(doc):
                    page = doc[page_num]
                    page_size = page.rect
                    crop_rect = fitz.Rect(
                        page_size.width * x_start, page_size.height * y_start,
                        page_size.width * x_end, page_size.height * y_end
                    )
                    pix = page.get_pixmap(dpi=dpi, colorspace=fitz.csRGB, clip=crop_rect)
                    output_path = os.path.join(output_folder, f"{pdf_name}_page_{page_num + 1}_cropped.png")
                    pix.save(output_path)
            
                    # **이미지 대비 강화 (Pillow 사용)**
                    # 저장된 이미지를 읽고 대비를 강화
                    image = Image.open(output_path)
                    # 대비 및 선명도 강화
                    contrast_enhancer = ImageEnhance.Contrast(image)
                    image = contrast_enhancer.enhance(1.8)  # 대비 강화
                    sharpness_enhancer = ImageEnhance.Sharpness(image)
                    image = sharpness_enhancer.enhance(2.0)  # 선명도 강화
            
                    # 강화된 이미지 저장
                    image.save(output_path)
        
        messagebox.showinfo("완료", f"이미지 추출 완료!\n저장 폴더: {output_folder}")
    except Exception as e:
        messagebox.showerror("오류", f"오류 발생: {str(e)}")
tk.Button(root, text="PDF 이미지 추출", command=extract_images).grid(row=7, column=0, columnspan=2, pady=10)

# Tkinter GUI 실행
root.mainloop()


In [9]:
# 각 페이지에서 텍스트 추출
for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()
    print(f"Page {page_num + 1}:\n{text}\n")
print("텍스트 추출 완료!")

Page 1:
시스템 가새(Ø42.7) 가새 _ 상세"B"
P17
단관파이프(Ø48.6) 가새 _ 상세 "C"
수평연결재(Ø48.6) _ 상세 "D"
벽연결철물 _ 상세 "E,F"
<경사재방향>
도면번호
<경사재(Φ42.7X2.3T)>
B1715(2139㎜)
거푸집 및 시스템 동바리 구조검토
청평3지구 도시개발사업 공동주택 신축공사
설계자 : 이민영
검토자 : 서상혁
피노엔지니어링(주)
서울시 서초구 서초동 1471-7
B1715
B1712
B1709
B1215
B1212
B1209
P17
P17
P17
1.수직재
P17
1,725
1291
863
432
216
3.Size
No.
P17
P12
P08
P04
P02
Size(mm)
1725
H15
H12
H09
H06
H03
NOTE
*멍에재는 아래와 같이 사선방향으로 
(가운데 멍에재 쐐기목 설치)
배치하여 편심하중이 발생되지 않도록 조치함.
2.수평재
H15
H12
H9
H6
H3
H18
Φ60.5mmx2.6t(SGT355)
Φ42.7mmx2.3t(SGT275)
Φ42.7mmx2.3t(SGT275)
Φ48.6mmx2.3t(SGT355)
수직재
수평재
경사재
수평연결재
사용 부재
검토부재
P12
P8
P4
P2
1291
863
432
216
Size(mm)
No.
1524
1219
914
610
305
B1712(1974㎜) B1709(1845㎜) B1215(1809㎜) B1212(1610㎜) B1212(1610㎜)
@
150
@
305
@
100
@
914
@
400
@
610
@
914
거푸집 Panel
보
측
벽
∅60.5 x 2.6t
장 선 재
1차 멍 에 재
수 직 재
2차 멍 에 재
수직재2(상/하받침)
수 평 재
경 사 재
경사재(강관파이프)
프레임(Frame)
거푸집 Panel
프레임(Frame)
플렛타이(Flat Tie)
□X50X50X2.3t
□X125X75X3.2t
T = 12mm 합판
84x84 미송각재
∅48.6 x 3.2t
∅42.7 x 2.3t
∅48.6 x 2.3t
∅42.7 x